In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#결과 화면에서 테이블을 크게 보기 위한 설정
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100) 

### 특성 테이블 정리

In [4]:
df_char = pd.read_excel('./최종자료/최종_특성_테이블.xlsx')

In [5]:
df_char.head()

,질병명,견종,나이,나이(꼬리_기준),성별,크기
0,갑상선기능저하증(Hypothyroidism),골든 리트리버,"4,10","성견,노견","중수, 암",대
1,갑상선기능저하증(Hypothyroidism),도베르만 핀셔,"4,10","성견,노견","중수, 암",대
2,갑상선기능저하증(Hypothyroidism),아이리쉬 세터,"4,10","성견,노견","중수, 암",대
3,갑상선기능저하증(Hypothyroidism),그레이트 덴,"4,10","성견,노견","중수, 암",대
4,갑상선기능저하증(Hypothyroidism),에어델 테리어,"4,10","성견,노견","중수, 암",대


In [6]:
df_char = df_char.drop(columns=['나이'])

In [7]:
df_char.head()

,질병명,견종,나이(꼬리_기준),성별,크기
0,갑상선기능저하증(Hypothyroidism),골든 리트리버,"성견,노견","중수, 암",대
1,갑상선기능저하증(Hypothyroidism),도베르만 핀셔,"성견,노견","중수, 암",대
2,갑상선기능저하증(Hypothyroidism),아이리쉬 세터,"성견,노견","중수, 암",대
3,갑상선기능저하증(Hypothyroidism),그레이트 덴,"성견,노견","중수, 암",대
4,갑상선기능저하증(Hypothyroidism),에어델 테리어,"성견,노견","중수, 암",대


In [8]:
df_char.rename(columns={'나이(꼬리_기준)':'나이'}, inplace=True)

In [9]:
df_char.head()

,질병명,견종,나이,성별,크기
0,갑상선기능저하증(Hypothyroidism),골든 리트리버,"성견,노견","중수, 암",대
1,갑상선기능저하증(Hypothyroidism),도베르만 핀셔,"성견,노견","중수, 암",대
2,갑상선기능저하증(Hypothyroidism),아이리쉬 세터,"성견,노견","중수, 암",대
3,갑상선기능저하증(Hypothyroidism),그레이트 덴,"성견,노견","중수, 암",대
4,갑상선기능저하증(Hypothyroidism),에어델 테리어,"성견,노견","중수, 암",대


In [10]:
df_char = df_char.dropna(axis=0) #결측 값이 존재하는 행 제거

In [11]:
# 다른 함수들이 참조할 수 있겠끔 global로 설정
breed = None
cate_age = None
gender = None
size = None

In [12]:
# 입력 받은 나이를 나이 카테고리에 맞게 변환해주는 함수
def change_age(age):
    
    if age[-1]=='살':
        if int(age[:-1]) >=8:
            return "노견"
        else :
            return "성견"
        
    elif age[-1]=='주':
        return "신생견"
        
    elif age[-2:]=='개월':
        return "자견"
    else:
        return '전연령'
    

In [13]:
#원하는 문자열이 row라는 큰 문자열 내에 존재하는지 검사해주는 함수
def check_range(row, age_list):
    new_row = [val.strip() for val in row.split(',')]
    for sub_age in age_list:
        if sub_age in new_row:
            return True

    return False
    

In [14]:
# 각 특성을 입력받고 특성 테이블에서 이에 해당하는 로우들로 새로운 테이블을 만든다.

def make_table_disease(df_char):
    global breed, cate_age, gender, size 
    
    #입력 받는 부분
    breed = input("견종을 입력해주세요. : (꼬리 앱 기준의 정확한 명칭의 견종 or 모르시겠다면 '무관'을 입력해주세요.) : ")
    display(breed)
    age = input("나이를 입력해주세요 : (몇 주라면 몇주를(단 1개월 이상이라면 개월을 써주세요), 개월의 경우 숫자 뒤에 개월을(단 12개월 이상이라면 나이로써 대해주세요), 나이라면 뒤에 살이라고 써주세요.) : ")
    cate_age = change_age(age)
    display(cate_age)
    gender = input("성별을 입력해주세요 : (중암(중성화 수술을 한 암컷), 중수(중성화 수술을 한 수컷), 암, 수)로 입력을 해주세요 : ")
    display(gender)
    size = input("본인의 개가 대형견이라면 대를, 중형견이라면 중을, 소형견이라면 소를 입력해주세요 : ")
    display(size)
    
    #크기 처리
    df_sub_char = df_char[(df_char['크기']==size) | (df_char['크기']=='무관')]
    
    #견종처리
    df_sub_char = df_char[df_char['견종'].isin([breed, '무관'])]

    #나이 처리
    df_sub_char['검사'] = [False]*len(df_sub_char)
    display(df_sub_char.head())
    df_sub_char['검사'] = df_sub_char.apply(lambda row : check_range(row['나이'], [cate_age,'전연령']), axis=1)
    display(df_sub_char.head(15))
    
    remove_list = df_sub_char[df_sub_char['검사']==False].index.tolist()
    df_sub_char = df_sub_char.drop(remove_list)
    df_sub_char = df_sub_char.drop(columns=['검사'])
    
    display(df_sub_char.shape)
    
    #성별 처리
    df_sub_char['검사'] = [False]*len(df_sub_char)
    display(df_sub_char.head())
    df_sub_char['검사'] = df_sub_char.apply(lambda row : check_range(row['성별'], [gender, '무관']), axis=1)
    display(df_sub_char.head(15))
    
    remove_list = df_sub_char[df_sub_char['검사']==False].index.tolist()
    df_sub_char = df_sub_char.drop(remove_list)
    df_sub_char = df_sub_char.drop(columns=['검사'])
    
    display(df_sub_char.shape)
    
    return df_sub_char
    

### 치료 테이블과 조인

In [15]:
table_cure = pd.read_excel(r'./최종자료/질병 치료.xlsx')

In [16]:
table_cure = table_cure.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
table_cure.columns = ['질병명', '치료요소', '질병코드']

In [17]:
def combination(x):
    if x == np.nan:
        return x
    c_list = []
    for val in str(x).split(','):
        c_list.append(val.strip())
    return ','.join(c_list)

In [18]:
table_cure['질병코드'] = table_cure['질병코드'].map(lambda x: combination(x))

In [19]:
# 질병 테이블과 입력 받은 특성들의 조건에 맞는 특성 테이블의 로우들을 조인해주는 함수

def make_perfect_table():
    first_table = make_table_disease(df_char) #특성 테이블에서 입력한 값들을 통해서 행들
    
    first_table_index = first_table['질병명'].tolist()
    second_table = table_cure[table_cure['질병명'].isin(first_table_index)]
    
    perfect_table = pd.merge(first_table, second_table, on="질병명")
    display(perfect_table.head(), perfect_table.shape)
    
    return perfect_table



# 증상 추가하기

In [20]:
df_all = pd.read_excel('./최종자료/diseas_result5.xlsx')

### 증상

In [21]:
# 증상을 입력받고 질병 테이블에서 가장 의심되는 질병 3개를 추천해주는 함수

def recomended() :
    state_list = []
    disease_data = None
    pet = '개'
    emer = {'응급':1, np.nan:0}
    
    #while 문을 통해서 증상을 계속해서 입력받는다.
    while 1 :
        state = input('증상을 입력 해주세요 . :').rstrip() 
        
        # 문자 'q'를 입력 받았을때 아래의 부분을 처리하고 while문을 탈출합니다.
        if state == 'q':
            # 단 하나의 증상을 입력했을 경우
            print('\n')
            print('가장 의심되는 질병 진단 : \n')
            
            if len(state_list)==1:
                #응급도 컬럼의 내용을 숫자로 변경합니다.
                disease_data['응급도'] = disease_data['응급도'].map(lambda x: emer[x]) 
                #질병 테이블을 빌병 발생빈도, 응급도 컬럼을 기준으로 해서 내림차 정렬을 합니다.
                disease_data = disease_data.sort_values(by=['질병발생빈도', '응급도'], ascending=False)

                return disease_data.head(3)
            
            else:
                
                # 복수의 증상을 입력했을 경우
                size = len(state_list) #입력받은 증상의 갯수를 보관하는 변수
                
                #현재 disease_data라는 데이터 프레임은 고객이 입력한 증상들을 갖는 질병들로 구성되어 있다. 증상이 하나만 일치하거나
                #입력한 증상이 모두 일치하는 그런 질병들로 구성됨
                #사용자가 입력한 증상을 제일 많이 가지고 있는 질병이 몇개인지
                #단, 여기서 증상의 갯수만큼 존재하지는 않는다. 축종에 의해서 축종이 일치하거나 공통이라는 카테고리가 있기 때문에
                #max_count는 증상의 갯수 이상일 것이다.
                max_count = disease_data.groupby('질병명').count().max().max() 
                
                
                plus_val = 0
                
                if max_count >= size: #입력한 증상을 최대한 많이 갖는 질병들만 발생 빈도를 높여준다. 즉 가중치 값을 올려준다.
                    
                    disease_count = disease_data.groupby('질병명').count().reset_index()
                    disease_max = disease_data.groupby('질병명')['질병발생빈도'].max().reset_index()

                    #입력한 증상들을 많이 갖는 질병들 중에서 발생빈도수가 제일 많은 질병의 발생 빈도수를 저장한다.
                    plus_val = disease_max['질병발생빈도'].max()
                    
                    # 증상들을 많이 갖는 질병들을 리스트로 만든다.
                    high_probability_of_disease = disease_count[disease_count['질병코드']>=size]['질병명'].tolist() 
                    
                    #증상들을 많이 갖는 질병들의 질병 발생 빈도수를 질병 발생빈도수가 제일 큰 값만큼 올려준다.(가중치 up)
                    disease_data['질병발생빈도'] = disease_data.apply(lambda row : row['질병발생빈도']+plus_val if row['질병명'] in high_probability_of_disease else row['질병발생빈도'], axis=1)
                
                # 축종이 공통인 부분 보다는 축종이 일치하는 발생빈도수를 2배로 증가 시켜준다.(가중치 up)
                disease_data.loc[(disease_data['질병축종']==pet),'질병발생빈도'] = disease_data.loc[(disease_data['질병축종']==pet),'질병발생빈도']*2
                disease_data['응급도'] = disease_data['응급도'].map(lambda x: emer[x])
                
                # 질병명과, 질병코드를 기준으로 해서 합산 테이블을 만든다. 각 질병명을 기준으로 해서 질병발생빈도수, 응급도 값들을 더해진다.
                result = disease_data.groupby(['질병명', '질병코드'])['질병명', '질병코드', '질병발생빈도', '응급도'].sum().reset_index()
                
                #질병 발생 빈도와 응급도를 기준으로 내림차순으로 정렬한다.
                result = result.sort_values(by=['질병발생빈도', '응급도'], ascending=False)
                
                return result.head(3)
            
        else:   #'q'를 입력해 중단하지 않음 
            
            state_list.append(state)

            if len(state_list)==1: # 단 하나의 증상만 입력했을 경우

                df_result = df_all[(df_all['표준증상명']==state)]
                df_result = df_result[(df_all['질병축종']=='공통') | (df_all['질병축종']==pet)]
                disease_data = df_result
                print(df_result)
                
            else:
                # 복수의 증상을 입력 했을 때
                size = len(df_all)
                length = len(state_list)
                
                df_result = df_all.copy()
                df_result['확인'] = np.zeros(size)
                
                # 이제까지 입력한 증상들을 갖는 질병들만 고른다.
                df_result = df_all[df_all['표준증상명'].isin(state_list)] 
                
                #질병 축종이 공통이거나 개인 얘들만 고른다.
                df_result = df_result[(df_result['질병축종']=='공통') | (df_result['질병축종']==pet)]
                df_result = df_result.sort_values(by=['표준증상명'])
            
                # 아래의 경우는 본래 증상의 갯수만큼 혹은 그 이상의 행을 가진 질병에 대한 데이터만을 가지겠끔 하려고 했었는데
                # 그경우에 펫의 주인이 의심되는 대로 입력하다가 본래의 특정한 병에 접근이 불가능할 경우도 있기에
                # 과감히 그 기능은 빼고 위의 'q'를 입력시에 제일 의심이 가는 질병을 추천하겠끔 만들었습니다.
            
                disease_data = df_result
                print(df_result.head(15))
        
    return 0

### 가중치 선별

In [22]:
# 사용자가 입력한 각 특성들에 가중치를 주는 함수입니다.
# 제일 중요하다고 생각하는 특성은 견종이며, 다음으로 나이, 크기, 성별 순입니다.

def score_row(row, val):
    
    score = 0
    
    global breed, cate_age, gender, size 
    
    if val=='견종':
        if row==breed:
            score += 10
        else : 
            score += 1
    
    elif val=='나이':
        
        if cate_age in row:
            score += 5
        else:
            score += 1
    
    elif val=='크기':
        if row==size:
            score += 3
        else:
            score += 1
    
    elif val=='성별':
        if row==gender:
            score+=2
        else:
            score+=1
    
    return score

### 가중치 총합에 따라 정렬

In [23]:
def sorted_perfect_table(perfect_table):
    perfect_table['총합'] = [0]*len(perfect_table)
    
    #질병에 대해 특성들의 가중치의 합을 '총합'컬럼에 입력해줍니다.
    perfect_table['총합'] = perfect_table.apply(lambda row : score_row(row['견종'], '견종')+score_row(row['나이'], '나이')+score_row(row['성별'], '성별')+score_row(row['크기'], '크기'), axis=1)
    #가중치의 합을 기준으로 내림차순으로 정렬합니다.
    perfect_table = perfect_table.sort_values(by=['총합'], ascending=False)
    print("검사 통한 테이블")
    
    return perfect_table

### 증상을 통해 나온 3개의 질병 순위를 위한 함수

In [24]:
# recommended 함수를 통해서 전달받은 질병들에는 리스트 내에서 앞에 있는 순으로 우선 순위가 존재합니다.
# 특성 들에 의한 데이터 선별보다는 과거 펫이 겪었던 질병이력이 더 주목해야할 데이터 이므로
# 정렬시 이 순위를 우선시 해줍니다.
# 단, 정렬시 가중치합이 내림차순이므로 이에 맞추기위해 본래 0,1,2 순이었던 데이터를 3,2,1 순으로 변경해줍니다.

def set_rank(row, code_list):
    score = 0
    new_row = [val.strip() for val in row.split(',')]
    for sub_code in code_list:
        if sub_code in new_row:
            if score < code_list[sub_code]:
                score = code_list[sub_code]
    return score

### 중간 단계의 테이블을 만들기 위한 함수(사료테이블과 조인 전의 중간 단계의 테이블이라는 의미이다.)

In [25]:

def confirm_middle_stage():
    perfect_table = make_perfect_table()
    
    #특성을 통해 얻어온 조인된 테이블을 가중치 합을 통해 정렬시켜줍니다.
    perfect_table = sorted_perfect_table(perfect_table)
    display(perfect_table.head(7))
    
    if input('과거 병력중 증상을 입력할 건가요?')=='yes':
        
        # 증상을 통해 가장 의심되는 질병 데이터를 얻어 옵니다.
        symptom = recomended()
        print("출력")
        symptom = symptom.iloc[0:3,:].reset_index()
        symptom.rename(columns={0:'질병명', 1:'질병코드', 2:'질병발생빈도', 3:'응급도'}, inplace=True)
        display(symptom)
        symptom = symptom.drop(columns=['index'])

        disease_code = symptom.iloc[0:3,1]
        display(disease_code)
        
        dict_disease_code = {val:(3-idx) for idx, val in enumerate(disease_code)} #우선 순위값을 내림차순으로 정렬하기 위해 변경해줍니다.

        print("질병 데이터를 넣기 전 테이블")
        display(perfect_table.head())
        
        # 특성, 치료의 조인된 테이블내에 가장 의심되는 질병의 질병 코드가 존재하는지 검사합니다.
        perfect_table['검사'] = [False]*len(perfect_table)
        perfect_table['검사'] = perfect_table.apply(lambda row: check_range(row['질병코드'], disease_code), axis=1) 
        
        print("트루 로우 갯수 : ")
        display(len(perfect_table[perfect_table['검사']==True]))
        
        if len(perfect_table[perfect_table['검사']==True]) != 0:
            
            # 가장 의심되는 질병의 질병코드가 조인된 테이블 내에 하나도 존재하지 않는 경우
            
            perfect_table = perfect_table[perfect_table['검사']==True] # 질병코드가 존재하는 열들만 골라냅니다.
            perfect_table['순위'] = [0]*len(perfect_table)
            
            #가장 의심되는 질병들의 코드가 존재하는 열들에 대해 이 질병들의 우선 순위에 대해서 '순위'라는 컬럼에 입력해 줍니다.
            perfect_table['순위'] = perfect_table.apply(lambda row : set_rank(row['질병코드'], dict_disease_code), axis=1)
            # 순위와 총합을 기준으로 데이터를 내림차순으로 정렬해줍니다.
            perfect_table = perfect_table.sort_values(by=['순위', '총합'], ascending=False)
        
        perfect_table = perfect_table.drop(columns=['검사'])

        display(perfect_table)

        return perfect_table
    
    else:
        
        # 증상을 입력하지 않는다면 위의 경우처럼 데이터 프레임으로 전달하는 것이 아니라 스트링(문자열)로 전달을 합니다.
        # 단, 이는 견종이 '무관'일 경우 입니다.
        
        if breed == '무관':
            
            if cate_age=='전연령':
                return "전연령"
            elif cate_age == '신생견' or cate_age == '자견':
                return '자견'
            elif cate_age == '성견':
                return '성견'
            else:
                return '노견'
        else:
            return perfect_table
            

# 사료 테이블과 조인 & 사료 추천

In [26]:
df_goods = pd.read_excel("./최종자료/고려대전달용_쇼핑몰_공개_상품목록_20200814.xlsx")

In [27]:
df_goods.columns = df_goods.iloc[0,:]

In [28]:
df_goods = df_goods.drop([0], axis=0)

In [29]:
df_goods['주요기능'] = ['']*len(df_goods)

### 사료테이블 내의 나누어진 치유 기능 ',' 이용해서 합치기

In [30]:
def make_func(row):
    
    func_list = []
    
    if row['주요기능1'] is not np.nan:

        func_list.append(str(row['주요기능1']))
    
    if row['주요기능2'] is not np.nan:

        func_list.append(str(row['주요기능2']))
    
    if row['주요기능3'] is not np.nan:

        func_list.append(str(row['주요기능3']))
    
    if row['주요기능4'] is not np.nan:

        func_list.append(str(row['주요기능4']))
        
    if row['주요기능5'] is not np.nan:

        func_list.append(str(row['주요기능5']))
        
    if row['주요기능6'] is not np.nan:

        func_list.append(str(row['주요기능6']))
        
    return ",".join(func_list)

In [31]:
df_goods['주요기능'] = df_goods.apply(lambda row : make_func(row), axis=1)

### 앞에서 넘어온 치료리스트와 사료테이블 내의 치료 리스트가 얼마나 일치하는가에 대한 함수

In [32]:
def make_match(row, goods_list):
    set_A = set(row['주요기능'].split(','))
    set_B = set(goods_list[0].split(','))

    return len(set_A & set_B)

### 견종이 무관인 경우의 각 치료요소 리스트 만드는 함수

In [33]:
def get_top_three(answer):
    
    cure_list = []
    
    if answer == '전연령':
        cure_list = ['피부(모질)개선','귓속 건강','설사 예방']
    elif answer == '자견':
        cure_list = ['설사 예방','영양공급','면역력 향상']
    elif answer == '성견':
        cure_list = ['설사 예방','귓속 건강','피부(모질)개선']
    elif answer == '노견':
        cure_list = ['피부(모질)개선','귓속 건강','관절']
    
    return cure_list

# 최종적으로 사료를 추천해주는 함수

In [34]:
def recommend_goods(df_goods):
    
    goods = input("찾는 것은 SNACK(과자)인가요? 아니면 FEED(사료)인가요?")
    
    middle_table = confirm_middle_stage() # 질병 코드 처리까지 마쳤거나 그 이전의 테이블을 리턴 받습니다.
    
    #견종이 무관인지 검사
    if type(middle_table) == type('exit'):
        answer = middle_table
        cure_list = get_top_three(answer) #견종이 무관인 경우 치료리스트를 통해서 바로 사료 테이블과 조인하기위해 견종과 나이에 해당하는 치료요소를 리턴 받스빈다.
        display(cure_list)
    
    else:
        
        first = pd.DataFrame(middle_table.iloc[0,:]).T
        display(first)
    
        cure_list = first['치료요소'].tolist()
        display(cure_list)
    
    goods_list = df_goods[(df_goods['축종구분']=='개') & (df_goods['큐레이션 타입']==goods)]
    display(goods_list.shape)

    # '일치'라는 컬럼을 새로 만들고, 위의 치료요소 리스트와 사료 테이블내의 치료요소와 얼마난 일치하는지 수치로 입력합니다.
    goods_list['일치'] = goods_list.apply(lambda row: make_match(row, cure_list), axis=1) 
    
    # '일치' 컬럼을 통해서 많이 일치하는 순으로 데이터를 내림차순으로 정렬합니다.
    goods_list = goods_list.sort_values(by=['일치'], ascending=False)
    
    display(goods_list.head(20))
    display(goods_list['일치'].value_counts())
    
recommend_goods(df_goods)

찾는 것은 SNACK(과자)인가요? 아니면 FEED(사료)인가요?FEED
견종을 입력해주세요. : (꼬리 앱 기준의 정확한 명칭의 견종 or 모르시겠다면 '무관'을 입력해주세요.) : 블랙 테리어


'블랙 테리어'

나이를 입력해주세요 : (몇 주라면 몇주를(단 1개월 이상이라면 개월을 써주세요), 개월의 경우 숫자 뒤에 개월을(단 12개월 이상이라면 나이로써 대해주세요), 나이라면 뒤에 살이라고 써주세요.) : 3살


'성견'

성별을 입력해주세요 : (중암(중성화 수술을 한 암컷), 중수(중성화 수술을 한 수컷), 암, 수)로 입력을 해주세요 : 수


...

,질병명,견종,나이,성별,크기,치료요소,질병코드,총합,순위
23,"외이도염, 중이염(Otitis externa and media)",블랙 테리어,전연령,무관,초대,"귓속 건강,면역력 향상","DIS0178,DIS0228",13,3


['귓속 건강,면역력 향상']

(403, 33)

,상품코드,상품명(*),카테고리 코드,카테고리,제조사,브랜드코드,원산지,상품 상세 설명(모바일),큐레이션 타입,축종구분,주요기능1,주요기능2,주요기능3,주요기능4,주요기능5,주요기능6,추천연령,추천품종타입,주원료1,주원료2,주원료3,주원료4,주원료5,조단백질(%),조지방(%),칼슘(%),수분(%),조섬유(%),조회분(%),인(%),함유성분,원료구성,주요기능,일치
947,G2000000577,지니펫 더 홀리스틱 홍삼&호주산 양고기,dog-feed1,강아지 > 사료 > 건식사료,지니펫,80,한국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,눈 건강,면역력 향상,영양공급,NaN,NaN,NaN,강아지 전연령,무관,고구마,양,해바라기씨,아마씨,홍삼,21,18,0.9,12,6,8,0.4,"조단백 21.0% 이상,조지방 18.0% 이상,조섬유 6.0% 이하,조회분 8.0%...","고구마분,양고기분,가수분해양고기,계유,유기농해바라기씨박,대두유,락토프리치즈분말,아마...","눈 건강,면역력 향상,영양공급",1
1083,G2000000276,아보덤 그레인프리 치킨&베지터블,dog-feed1,강아지 > 사료 > 건식사료,아보덤,55,미국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,관절,뼈 강화,면역력 향상,알러지예방,피부(모질)개선,NaN,강아지 전연령,무관,닭,완두콩,PEA 전분,타피오카,NaN,27,9,0.9,10,9,0,0.6,"조단백질 27.0% 이상,조지방 9.0% 이상,조섬유 9.0% 이하,칼슘 0.9% ...","닭고기분말, 완두, 닭연골(콘드로이틴과 글루코사민 소스), 완두분말, 이집트콩, 타...","관절,뼈 강화,면역력 향상,알러지예방,피부(모질)개선",1
1079,G2000000296,아보덤 피너클 칠면조&펌킨,dog-feed1,강아지 > 사료 > 건식사료,피너클,56,미국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,면역력 향상,알러지예방,NaN,NaN,NaN,NaN,강아지 전연령,무관,칠면조,완두콩,퀴노아씨,호박,NaN,26,11,0.8,8,10,0,0.9,"조단백질 26.0% 이상,조지방 11.0% 이상,조섬유 10.0% 이하,칼슘 0.8...","칠면조, 칠면조분말, 완두, 완두분말, 이집트콩, 호박, 카놀라유, 토마토, 알팔파...","면역력 향상,알러지예방",1
1076,G2000000319,퓨리나 프로플랜 토이 베이비독,dog-feed1,강아지 > 사료 > 건식사료,퓨리나,35,미국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,두뇌 성장,면역력 향상,소화력 증진,영양공급,피부(모질)개선,NaN,강아지 1세 미만,소형,닭,옥수수글루텐밀,쌀,옥수수,비타민E,31,20,1.1,0,3,10,1.6,"조단백질 31% 이상,조지방 20% 이상,조섬유 3% 이하,조회분 10% 이하,칼슘...","순살 닭고기,옥수수 글루텐밀,양조쌀,통옥수수,혼합토코페롤(비타민 E형태)로 보존된 ...","두뇌 성장,면역력 향상,소화력 증진,영양공급,피부(모질)개선",1
1070,G2000000331,로얄캐닌 미디엄 퍼피,dog-feed1,강아지 > 사료 > 건식사료,로얄캐닌,15,프랑스,"<p><br style=""clear:both;"">&nbsp;<img src=""htt...",FEED,개,면역력 향상,소화력 증진,영양공급,NaN,NaN,NaN,강아지 1세 미만,중형,닭,돼지,옥수수,밀,사탕무과육,30,18,1.03,9.5,2.6,8.7,0.87,"조단백질 30.0% 이상,조지방 18.0% 이상,조섬유 2.6% 이하,조회분 8.7...","육분(닭, 칠면조, 오리), 동물성지방(가금류, 돼지), 옥수수, 밀글루텐, 사탕무...","면역력 향상,소화력 증진,영양공급",1
1062,G2000000340,더독 닥터소프트 연어와 감자,dog-feed1,강아지 > 사료 > 건식사료,더독,19,한국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,관절,뼈 강화,면역력 향상,변냄새 개선,소화력 증진,피부(모질)개선,강아지 전연령,무관,연어,옥수수,닭,소맥전분,NaN,25,12,0.5,0,5,10,0.5,"조단백 25.0%이상, 조지방 12.0%이상, 조섬유 5.0%이하, 조회분 10.0...","연어분, 대두박, 소맥분, 옥수수분, 닭고기분, 감자분말, 정제닭기름, 글리세린, ...","관절,뼈 강화,면역력 향상,변냄새 개선,소화력 증진,피부(모질)개선",1
1053,G2000000351,퓨리나 프로플랜 퍼포먼스 올라이프 30/20,dog-feed1,강아지 > 사료 > 건식사료,퓨리나,35,미국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,관절,뼈 강화,근력 강화,면역력 향상,소화력 증진,퍼포먼스,강아지 전연령,무관,닭,옥수수글루텐밀,쌀,비타민E,옥수수,30,20,0.9,12,3,0,0.7,"조단백질 30% 이상,조지방 20% 이상,조섬유 3% 이하,칼슘 0.9% 이상,인 ...","닭고기,옥수수 글루텐밀,양조쌀,혼합토코페롤(비타민 E형태)로 보존된 동물성 지방,천...","관절,뼈 강화,근력 강화,면역력 향상,소화력 증진,퍼포먼스",1
1047,G2000000372,ANF 램28 프리미엄 전연령 애견사료,dog-feed1,강아지 > 사료 > 건식사료,ANF,4,호주,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,관절,뼈 강화,면역력 향상,소화력 증진,영양공급,피부(모질)개선,강아지 전연령,무관,양,쌀,닭,효모 추출물,생선,28,12,1.45,10,4.5,11,0.8,"조단백질 28.0% 이상,조지방 12.0% 이상,조섬유 4.5% 이하,조회분 11....","양고기,곡분,쌀,쌀겨,닭지방,양고기향미제,건조맥주효모,소금,생선오일,키토산,비타민/...","관절,뼈 강화,면역력 향상,소화력 증진,영양공급,피부(모질)개선",1
876,G2000000758,네이처스 버라이어티 인스팅트 LID 오리고기,dog-feed1,강아지 > 사료 > 건식사료,네이처스 버라이어티,120,미국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,면역력 향상,소화력 증진,영양공급,피부(모질)개선,NaN,NaN,강아지 전연령,무관,오리고기,완두콩,타피오카,카놀라유,천연착향료,27,17,2.3,10,3,0,1.3,"조단백 27.0%이상, 조지방 17%이상, 조섬유 3%이하, 수분 10.0%이하, ...","오리육분, 완두콩, 타피오카, 카놀라오일, 오리고기, 몬모릴로나이트점토, 천연향료,...","면역력 향상,소화력 증진,영양공급,피부(모질)개선",1
877,G2000000755,네이처스 버라이어티 인스팅트 치킨,dog-feed1,강아지 > 사료 > 건식사료,네이처스 버라이어티,120,미국,"<p><img src=""https://shopres.cori.pet/upload/c...",FEED,개,면역력 향상,소화력 증진,영양공급,피부(모질)개선,NaN,NaN,강아지 전연령,무관,닭,칠면조,완두콩,병아리콩,연어,37,20,2.3,10,3,0,1.4,"조단백 37%이상, 조지방 20%이상, 조섬유 3%이하, 수분 10%이하, 칼슘 2...","닭고기, 칠면조고기밀, 닭고기밀, 완두콩, 병아리콩, 닭지방, 연어밀, 달걀, 타피...","면역력 향상,소화력 증진,영양공급,피부(모질)개선",1


0    278
1    125
Name: 일치, dtype: int64

In [35]:
# 귀에서 피가 남, 귀를 긁음, 머리를 기울임